# 데이터 전처리
1. '::' 별로 데이터가 구분되어 있어 이를 기준으로 데이터를 파싱한다.
2. 3점 이상 평가를 받은 영화만 남겨서 추천 후보의 질을 높인다. 
3. rating 컬럼 이름 count로 바꾸기 -> 이거 노드에 이렇게 하라고 되어있는데 이거 >>개선좀<< 해주세요...
`ratings.count`로 쓰면 dataframe내의 내장 속성과 겹쳐서 에러나요. 
4. movie id만 있으면 무슨 영화인지 모르니까 영화 자체의 속성을 담고 있는 데이터를 따로 로딩한다. -> 이번 노드에서 제일 헷갈렸던 부분. 영화 정보를 얻기 위해서 존재하는 거지, 평점을 위해 존재하는 데이터가 아님!
5. 실제 사용할 정보만 남기기.

In [40]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

ratings['count']

# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()


orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


그리고 노드에서 영화 정보에 담긴 영화 수와 평가에 참여한 사용자 수를 체크해보래서 해봤다. 

In [41]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = movies['movie_id'].nunique()

print(ratings)

         user_id  movie_id  count  timestamp
0              1      1193      5  978300760
1              1       661      3  978302109
2              1       914      3  978301968
3              1      3408      4  978300275
4              1      2355      5  978824291
...          ...       ...    ...        ...
1000203     6040      1090      3  956715518
1000205     6040      1094      5  956704887
1000206     6040       562      5  956704746
1000207     6040      1096      4  956715648
1000208     6040      1097      4  956715569

[836478 rows x 4 columns]


학습에 사용할 칼럼만 남기고 나머지 칼럼은 날린다.

In [42]:
use_cols = ['user_id','movie_id','count']
ratings = ratings[use_cols]
user_unique = ratings['user_id'].unique()
movie_unique = movies['movie_id'].unique()

# 추천 시스템 베이스 만들기
1. '나'를 추가하기 -> 내 취향에 맞는 데이터를 실존하는 데이터 안에서 골라서 집어넣기. 

In [43]:
sorted_m = movies.sort_values(by='genre', ascending=True)
print(sorted_m.head(10))
my_fav_name = [movies.loc[3570]['title'],movies.loc[2879]['title'],movies.loc[2687]['title'],movies.loc[143]['title'],movies.loc[540]['title']]
my_fav_id = [movies.loc[3570]['movie_id'],movies.loc[2879]['movie_id'],movies.loc[2687]['movie_id'],movies.loc[143]['movie_id'],movies.loc[540]['movie_id']]
print(my_fav_name)

      movie_id                                title   genre
3570      3639  Man with the Golden Gun, The (1974)  Action
2687      2756         Wanted: Dead or Alive (1987)  Action
1400      1424                        Inside (1996)  Action
143        145                      Bad Boys (1995)  Action
540        544             Striking Distance (1993)  Action
2896      2965               Omega Code, The (1999)  Action
248        251                   Hunted, The (1995)  Action
2920      2989            For Your Eyes Only (1981)  Action
2880      2949                        Dr. No (1962)  Action
2879      2948         From Russia with Love (1963)  Action
['Man with the Golden Gun, The (1974)', 'From Russia with Love (1963)', 'Wanted: Dead or Alive (1987)', 'Bad Boys (1995)', 'Striking Distance (1993)']


추천 시스템을 위해서 일관된 취향을 가지려고 데이터를 genre 순으로 정렬했더니 역시 한 분야의 영화들이 몰려서 나온다. 이 중에 상위 5개의 영화를 가지고 내 취향을 만든다. 

In [44]:
my_fav_list= pd.DataFrame({'user_id' : [6041]*5, 'movie_id' : my_fav_id, 'count' : [4,5,4,5,4]})
ratings = ratings.append(my_fav_list)
print(ratings.tail(10))

         user_id  movie_id  count
1000203     6040      1090      3
1000205     6040      1094      5
1000206     6040       562      5
1000207     6040      1096      4
1000208     6040      1097      4
0           6041      3639      4
1           6041      2948      5
2           6041      2756      4
3           6041       145      5
4           6041       544      4


요렇게 추가된 모습을 볼 수 있다. 여담이지만 옆에 index가 0부터 시작하는게 거슬리면 이를 리셋하는 코드가 있다. 
`df.reset_index()`

# 추천 시스템 만들기
1. Matrix Factorization을 위한 행렬 크기 잘 계산하기(여기서 매우 헷갈렸음)
이 과정은 ratings 데이터만 가지고 해도 충분하다! 근데 ratings에...index로 쓸 것이 있을까? 
중요한건 ratings 내의 총 영화수(중복 제거시) != 영화 정보 내의 총 영화수(중복 제거시) 이다. 
그래서 MF를 하려면 ratings내에서 item의 구분을 위해 따로 index를 만들어 줘야 한다. ratings내에서만 쓰일 index를 만드는 것이다. 
이번 노드에서 쓰인 방법은 이 논문에서 발췌해온 것이라고. (http://yifanhu.net/PUB/cf.pdf)
2. CSR 매트릭스 만들기
1번을 잘 정의했다면 필요한 행렬의 크기를 잘 계산할 수 있을 것이다. 
3. MF 모델 학습하기

In [45]:
from scipy.sparse import csr_matrix
num_user = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()
print(num_user,num_movies)

print("<데이터 분석>")
total_movies = movies['movie_id'].nunique()
print(total_movies, num_movies)

# 이 부분 코드가 매우 중요하다! user와 movie에 대한 인덱스를 만들어준다. 
# map 함수에 인자로 넣을 규칙.
user_to_uniq_idx = {v:k for k,v in enumerate(ratings['user_id'].unique())}
movie_to_uniq_idx = {v:k for k,v in enumerate(ratings['movie_id'].unique())}


# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_uniq_idx.get).dropna()
#print(temp_user_data)
#print(ratings.user_id)
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# ratings내의 movie에 대해서도 같은 작업을 해준다. 
temp_movie_data = ratings['movie_id'].map(movie_to_uniq_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id_uniq'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')


6040 3628
<데이터 분석>
3883 3628
user_id column indexing OK!!
movie column indexing OK!!


indexing을 따로 해주고 나서 그 후에 csr_matrix를 구할 수 있다. 
csr_matrix는 MF 행렬의 사이즈를 생각했을 때, 행렬이 너무 커지므로 이를 압축해서 올릴 수 있는 방법이 없을까 하여 생각해 낸 방법이다.0이 아닌 헹렬의 요소 위치를 기억하는 자료구조. 뭐여 sparse matrix였네. 
csr_matrix가 두 행렬으로 분해될 때, 이론은 다음과 같다. 
(전체 사용자의 특성 벡터가 모인 행렬) * (평가한 아이템들의 특성 벡터가 모인 행렬)
sparse matrix를 numpy에서 표현하는 방법은 여기를 참고하라. (https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005) 

In [46]:
csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id_uniq'])), shape=(num_user, num_movies))
print(csr_data)

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 665)	5
  (6039, 969)	4
  (6039, 2054)	5
  (6039, 2223)	4
  (6039, 3146)	4


In [47]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'


# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 추천받기

일단 영화에 대한 정보는 이렇게 찾는다. 
1. movies 데이터에서 찾으려는 영화의 '전체' index를 찾아낸다.
2. ratings의 movie_id와 user_id 컬럼은 자체 index를 가지고 있다. 1에서 얻은 index를 이용하자. 

In [51]:
#movie_name_search = {v:k for k,v in enumerate(movies['title'].unique())}
new_fav_movie = "From Russia with Love (1963)"
new_fav_movie_id = movies[movies['title'] == new_fav_movie]
print(new_fav_movie_id)

      movie_id                         title   genre
2879      2948  From Russia with Love (1963)  Action


In [53]:
print(new_fav_movie_id.values.tolist()[0])
new_fav_movie_id = new_fav_movie_id.values.tolist()[0][0]
new_fav_movie_id_in_ratings = movie_to_uniq_idx[new_fav_movie_id]
similar_movies = als_model.similar_items(new_fav_movie_id_in_ratings, N=15)
similar_movies


[2948, 'From Russia with Love (1963)', 'Action']


[(665, 1.0),
 (199, 0.7662193),
 (932, 0.7521499),
 (1025, 0.7480388),
 (992, 0.6683175),
 (966, 0.66034156),
 (967, 0.64210385),
 (969, 0.63595873),
 (929, 0.5777972),
 (1026, 0.5135011),
 (991, 0.5128461),
 (1308, 0.44402543),
 (1010, 0.43092686),
 (409, 0.40667576),
 (3466, 0.39105555)]

이렇게 추천받은 영화를 역으로 돌려야 할 때가 왔다. ratings 내의 유일한 movie_id로 출력된 결과물을 다시 전체의 고유한 id로 돌리고, 그리고 제목을 출력하는 코드를 짜려고 한다. 

In [59]:
idx_to_g_idx = {v:k for k,v in movie_to_uniq_idx.items()}
recommand_list = [idx_to_g_idx[i[0]] for i in similar_movies]
print(recommand_list)

[2948, 2947, 2993, 2949, 2991, 3633, 3635, 3639, 2989, 2376, 2990, 2944, 3638, 524, 989]


이제 이 영화 제목들을 movies에서 찾으면 된다. 

In [63]:
result = pd.DataFrame()
for i in recommand_list :
    print(movies[movies.movie_id == i])
print(result)

      movie_id                         title   genre
2879      2948  From Russia with Love (1963)  Action
      movie_id              title   genre
2878      2947  Goldfinger (1964)  Action
      movie_id               title   genre
2924      2993  Thunderball (1965)  Action
      movie_id          title   genre
2880      2949  Dr. No (1962)  Action
      movie_id                    title   genre
2922      2991  Live and Let Die (1973)  Action
      movie_id                                   title   genre
3564      3633  On Her Majesty's Secret Service (1969)  Action
      movie_id                         title   genre
3566      3635  Spy Who Loved Me, The (1977)  Action
      movie_id                                title   genre
3570      3639  Man with the Golden Gun, The (1974)  Action
      movie_id                      title   genre
2920      2989  For Your Eyes Only (1981)  Action
      movie_id                     title   genre
2307      2376  View to a Kill, A (1985)  Action
  

# 후기
이거는 MF 행렬 계산하는게 너무 어려웠다. 하고 CSR은 하나의 자료를 저장하는 방식일 뿐, 행렬에서 원소가 가득 들어차 있다면 그냥 행렬을 쓰는게 낫다. 추천시스템의 경우 데이터가 쌓이면 쌓일수록 제대로 추천받을 수 있는 확률이 높아지겠다. 내가 추천받은 영화의 경우 밑으로 가면 갈수록 값이 0.3으로 떨어지고 장르도 다른 장르로 바뀌면서 추천시스템의 효력이 떨어졌다. 넷플릭스를 구독하고 있는데 내가 호기심에 딱 한번 공포영화를 클릭했더니, 계속 관련 장르를 추천해주는 것도 이러한 이유에서 벌어진 건가 싶다.